In [495]:
import os
import pandas as pd
import sklearn as sk
import  psycopg2
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
import warnings
import getpass
import numpy as np
import seaborn as sns
import joblib
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from scipy.optimize import minimize
import pyreadr
pd.set_option('display.max_columns', 50)
import redshift_connector
from sqlalchemy import create_engine
import plotly.express as px

from pandas.tseries.offsets import MonthEnd

In [496]:
password = getpass.getpass('Password: ')

In [576]:
conn = psycopg2.connect(dbname='n26'
                       , host = 'n26-dwh.cfxsmcyyfcch.eu-central-1.redshift.amazonaws.com'
                       , port = '5439'
                       , user= 'ilya_katliarou_reader'
                       , password= password
                       )

In [577]:
st = time.time()

query = f"""
select 
m.encoded_key
, mum.user_id
, mp.product
, m.loan_amount
, max(m.creation_date::date)::date as creation_date
from  mmbr_loan_account m 
inner join dbt.mmbr_user_match mum on mum.mmbr_client_key = m.account_holder_key
inner join dbt.mmbr_loan_product_mapping mp on m.loan_name = mp.loan_name and mp.product in ('consumer_credit')
where m.creation_date::date >= '2024-01-01'::date and m.creation_date::date <= '2025-02-28'
group by 1, 2, 3, 4
"""

data = pd.read_sql_query(query, conn)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time/60, 'minutes')

/var/folders/1f/pct68c2j21dd7jgky0y_thb80000gp/T/ipykernel_86402/117791520.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


Execution time: 1.383237318197886 minutes


In [578]:
data

,encoded_key,user_id,product,loan_amount,creation_date
0,8a868e01949502e701949ec9bcbc5269,405c64bf-1a1c-41ff-b6cc-88b889396def,consumer_credit,13000.0,2025-01-25
1,8a868e099175234a0191b87338ad46bd,e4e335a0-b4fb-4051-8abf-731ea856d904,consumer_credit,15000.0,2024-09-03
2,8a868e099175234a0191c0c5c6461301,a819125a-ea89-4345-8a47-5be0cc920fa7,consumer_credit,10000.0,2024-09-05
3,8a868e0a917523530191acdc39a724ac,9d96c0eb-ff0b-402c-b62a-83e3aea32daf,consumer_credit,15000.0,2024-09-01
4,8a868e1a8d0c965d018d11ad49823f4e,11e7737d-fc27-4863-8d1b-a6669445bfbe,consumer_credit,3000.0,2024-01-16
...,...,...,...,...,...
12348,8a868fe18e65d8dc018e6cad949d44df,8af0f60e-8605-4974-b60a-a214e262d33e,consumer_credit,3000.0,2024-03-23
12349,8a868ff0936f6afc01937c21de9178bf,082ec984-f6ff-4dce-addd-236fa63a1f4d,consumer_credit,4000.0,2024-11-30
12350,8a868ffb9175234e0191acccfc214206,20314e4f-1a79-468e-b3be-58d817b05ac4,consumer_credit,25000.0,2024-09-01
12351,8a868fff94dd10cf0194dde9bafa764a,255e8dd5-a750-446f-ba50-04f1ef06e800,consumer_credit,8000.0,2025-02-07


In [579]:
st = time.time()

query = f"""
select user_id, transaction_id as encoded_key, eur_written_off, credit_creation_date, max(write_off_dt::date)::date as write_off_dt 
from dbt.write_off
where reason = 'Credit'
and write_off_dt::date >= '2024-01-01'::date
group by 1, 2, 3, 4
"""

writeoff_data = pd.read_sql_query(query, conn)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time/60, 'minutes')

/var/folders/1f/pct68c2j21dd7jgky0y_thb80000gp/T/ipykernel_86402/1712837732.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  writeoff_data = pd.read_sql_query(query, conn)


Execution time: 0.12016273339589437 minutes


In [580]:
writeoff_data['eur_written_off'].sum()

2802955.0

In [591]:
writeoff_data['write_off_dt'] = pd.to_datetime(writeoff_data['write_off_dt'])
writeoff_data['credit_creation_date'] = pd.to_datetime(writeoff_data['credit_creation_date'])

#merged['creation_date_eom'] = merged['creation_date'] + MonthEnd(0)
writeoff_data['write_off_dt_eom'] = writeoff_data['write_off_dt'] + MonthEnd(0)

In [592]:
writeoff_data

,user_id,encoded_key,eur_written_off,credit_creation_date,write_off_dt,write_off_dt_eom
0,bb0d4afe-cb7f-43ef-87a0-cf61b7f073ad,8a868ea5926d6e1401927275d1543c1a,1939.0,2024-10-09 18:06:17,2024-12-13,2024-12-31
1,3ec15cdc-bdd6-4c13-8a64-2d0a4a462384,8a868ef482da202e0182db0673290f0e,4476.0,2022-08-26 16:40:09,2025-03-17,2025-03-31
2,4ad08b86-e430-4e83-b567-c21b3832f803,8a868fa5860e158701860eb3ce4c2a37,11041.0,2023-02-01 20:38:30,2025-03-17,2025-03-31
3,ca2efb43-e565-470c-8c8d-945d2297e5c0,8a868e909123d14a01912bf59cd70134,529.0,2024-08-07 08:30:02,2025-08-14,2025-08-31
4,632a4a56-a69c-443a-b146-2256dedbc048,8a868e8a856e3fb5018572a678b94e74,2674.0,2023-01-02 13:23:10,2024-03-07,2024-03-31
...,...,...,...,...,...,...
535,2e12b4e9-07e4-451b-9047-042cc223abb8,8a868ec98f89a983018f9cc7ec832741,729.0,2024-05-21 20:11:39,2025-06-16,2025-06-30
536,19ff1eb6-9cac-4dd0-ab4d-ea1382c0007f,8a868e74828ad81301828e89d56c0e07,2960.0,2022-08-11 20:13:17,2024-02-05,2024-02-29
537,7b3c6d99-e4c0-40ed-8c8d-f728b24b9a86,8a868e5f89795743018982a69a666d47,696.0,2023-07-23 12:08:23,2024-09-30,2024-09-30
538,149ab976-2473-4362-ab30-6be10b73c58e,8a868f1a8a42d331018a436e76f9464c,1567.0,2023-08-29 22:33:48,2024-10-28,2024-10-31


In [596]:
test = writeoff_data.merge(schufa_score_data, how='inner', on='user_id')

test_nop = test[test['worst_rating'].isin(['N','O','P'])]

In [598]:
test_nop.groupby(['write_off_dt_eom', 'worst_rating'])['eur_written_off'].sum().to_csv('split_by_nop.csv')

In [559]:
merged = pd.merge(
    data, 
    writeoff_data[['user_id', 'encoded_key', 'write_off_dt', 'eur_written_off']], 
    on=['user_id', 'encoded_key'], 
    how='left'
)

merged['creation_date_eom'] = merged['creation_date'] + MonthEnd(0)
merged['write_off_dt_eom'] = merged['write_off_dt'] + MonthEnd(0)

In [560]:
merged['writeoff_within_6m'] = (
    (merged['write_off_dt_eom'] >= merged['creation_date_eom']) &
    (merged['write_off_dt_eom'] <= merged['creation_date_eom'] + pd.DateOffset(months=6) + pd.DateOffset(days=3))
)
merged['eur_written_off_within_6m'] = merged['eur_written_off'].where(
    merged['writeoff_within_6m'], 0
)

In [561]:
merged

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m
0,8a868e099175234a0191d60079d965d2,e358f2ec-ccbd-4880-9bf0-d55086c5072f,consumer_credit,4000.0,2024-09-09,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0
1,8a868e13920c1a780192187747750bf7,6135ce49-c5f8-418d-b5ae-e6ac49708dc5,consumer_credit,3000.0,2024-09-22,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0
2,8a868e278dc345c9018dc86074f029c7,a7762b7c-1a1e-4c64-9947-d96a01ca498b,consumer_credit,1000.0,2024-02-20,NaT,NaN,2024-02-29 00:00:00,NaT,False,0.0
3,8a868e2a946bbfb0019474ab6d5132d3,6637abb5-8c4e-4efe-863a-4b3ff9ec3036,consumer_credit,1100.0,2025-01-17,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0
4,8a868e2a94f6d4180194f92d38636bb3,f41a88b3-ae36-4fc0-a5f0-f99ffe0de06e,consumer_credit,8000.0,2025-02-12,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
12348,8a868fe18e65d8dc018e6cad949d44df,8af0f60e-8605-4974-b60a-a214e262d33e,consumer_credit,3000.0,2024-03-23,NaT,NaN,2024-03-31 00:00:00,NaT,False,0.0
12349,8a868ff0936f6afc01937c21de9178bf,082ec984-f6ff-4dce-addd-236fa63a1f4d,consumer_credit,4000.0,2024-11-30,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0
12350,8a868ffb9175234e0191acccfc214206,20314e4f-1a79-468e-b3be-58d817b05ac4,consumer_credit,25000.0,2024-09-01,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0
12351,8a868fff94dd10cf0194dde9bafa764a,255e8dd5-a750-446f-ba50-04f1ef06e800,consumer_credit,8000.0,2025-02-07,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0


In [562]:
st = time.time()

query = f"""
select
a.encoded_key,
sum(r.interest_paid + r.fees_paid) as revenue_eur
from mmbr_loan_account as a
inner join mmbr_repayment as r
        on r.parent_account_key = a.encoded_key
            and r.repaid_date <= last_day(date_add('month', 6, creation_date::date))
inner join dbt.mmbr_loan_product_mapping lpm
        on a.loan_name = lpm.loan_name
            and lpm.product in ('consumer_credit')
group by 1
"""

revenue_data = pd.read_sql_query(query, conn)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time/60, 'minutes')

/var/folders/1f/pct68c2j21dd7jgky0y_thb80000gp/T/ipykernel_86402/2369637799.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  revenue_data = pd.read_sql_query(query, conn)


Execution time: 0.017949632803599038 minutes


In [563]:
merged_v1 = merged.merge(revenue_data, on=['encoded_key'], how='left')
merged_v1['revenue_eur'] = merged_v1['revenue_eur'].fillna(0)

In [564]:
merged_v1

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur
0,8a868e099175234a0191d60079d965d2,e358f2ec-ccbd-4880-9bf0-d55086c5072f,consumer_credit,4000.0,2024-09-09,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,204.71
1,8a868e13920c1a780192187747750bf7,6135ce49-c5f8-418d-b5ae-e6ac49708dc5,consumer_credit,3000.0,2024-09-22,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,80.73
2,8a868e278dc345c9018dc86074f029c7,a7762b7c-1a1e-4c64-9947-d96a01ca498b,consumer_credit,1000.0,2024-02-20,NaT,NaN,2024-02-29 00:00:00,NaT,False,0.0,42.16
3,8a868e2a946bbfb0019474ab6d5132d3,6637abb5-8c4e-4efe-863a-4b3ff9ec3036,consumer_credit,1100.0,2025-01-17,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,56.00
4,8a868e2a94f6d4180194f92d38636bb3,f41a88b3-ae36-4fc0-a5f0-f99ffe0de06e,consumer_credit,8000.0,2025-02-12,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,236.40
...,...,...,...,...,...,...,...,...,...,...,...,...
12348,8a868fe18e65d8dc018e6cad949d44df,8af0f60e-8605-4974-b60a-a214e262d33e,consumer_credit,3000.0,2024-03-23,NaT,NaN,2024-03-31 00:00:00,NaT,False,0.0,112.44
12349,8a868ff0936f6afc01937c21de9178bf,082ec984-f6ff-4dce-addd-236fa63a1f4d,consumer_credit,4000.0,2024-11-30,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,103.54
12350,8a868ffb9175234e0191acccfc214206,20314e4f-1a79-468e-b3be-58d817b05ac4,consumer_credit,25000.0,2024-09-01,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,1621.25
12351,8a868fff94dd10cf0194dde9bafa764a,255e8dd5-a750-446f-ba50-04f1ef06e800,consumer_credit,8000.0,2025-02-07,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,563.94


In [565]:
merged_v1['profit_eur'] = merged_v1['revenue_eur'] - merged_v1['eur_written_off_within_6m']

In [566]:
merged_v1

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur,profit_eur
0,8a868e099175234a0191d60079d965d2,e358f2ec-ccbd-4880-9bf0-d55086c5072f,consumer_credit,4000.0,2024-09-09,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,204.71,204.71
1,8a868e13920c1a780192187747750bf7,6135ce49-c5f8-418d-b5ae-e6ac49708dc5,consumer_credit,3000.0,2024-09-22,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,80.73,80.73
2,8a868e278dc345c9018dc86074f029c7,a7762b7c-1a1e-4c64-9947-d96a01ca498b,consumer_credit,1000.0,2024-02-20,NaT,NaN,2024-02-29 00:00:00,NaT,False,0.0,42.16,42.16
3,8a868e2a946bbfb0019474ab6d5132d3,6637abb5-8c4e-4efe-863a-4b3ff9ec3036,consumer_credit,1100.0,2025-01-17,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,56.00,56.00
4,8a868e2a94f6d4180194f92d38636bb3,f41a88b3-ae36-4fc0-a5f0-f99ffe0de06e,consumer_credit,8000.0,2025-02-12,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,236.40,236.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12348,8a868fe18e65d8dc018e6cad949d44df,8af0f60e-8605-4974-b60a-a214e262d33e,consumer_credit,3000.0,2024-03-23,NaT,NaN,2024-03-31 00:00:00,NaT,False,0.0,112.44,112.44
12349,8a868ff0936f6afc01937c21de9178bf,082ec984-f6ff-4dce-addd-236fa63a1f4d,consumer_credit,4000.0,2024-11-30,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,103.54,103.54
12350,8a868ffb9175234e0191acccfc214206,20314e4f-1a79-468e-b3be-58d817b05ac4,consumer_credit,25000.0,2024-09-01,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,1621.25,1621.25
12351,8a868fff94dd10cf0194dde9bafa764a,255e8dd5-a750-446f-ba50-04f1ef06e800,consumer_credit,8000.0,2025-02-07,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,563.94,563.94


In [567]:
merged_v1[merged_v1['writeoff_within_6m']==True]['eur_written_off_within_6m'].sum()

158235.0

In [568]:
merged_v1['revenue_eur'] = merged_v1['revenue_eur'].fillna(0)
merged_v1['profit_eur'] = merged_v1['profit_eur'].fillna(0)

In [583]:
st = time.time()

query = f"""

with rating_map as (
    select 'A' as rating, 1 as rating_rank union all
    select 'B', 2 union all
    select 'C', 3 union all
    select 'D', 4 union all
    select 'E', 5 union all
    select 'F', 6 union all
    select 'G', 7 union all
    select 'H', 8 union all
    select 'I', 9 union all
    select 'K', 10 union all
    select 'L', 11 union all
    select 'M', 12 union all
    select 'N', 13 union all
    select 'O', 14 union all
    select 'P', 15
),
scored as (
    select
        mum.user_id,
        m.encoded_key,
        c.rating,
        rm.rating_rank,
        c.purpose,
        m.loan_amount,
        c.requested_on::date as requested_on,
        m.creation_date::date as creation_date,
        row_number() over (
            partition by mum.user_id
            order by c.requested_on desc
        ) as rn_latest,
        row_number() over (
            partition by mum.user_id
            order by rm.rating_rank desc, c.requested_on desc
        ) as rn_worst
    from mmbr_loan_account m
    inner join dbt.mmbr_user_match mum 
        on mum.mmbr_client_key = m.account_holder_key
    inner join dbt.mmbr_loan_product_mapping mp 
        on m.loan_name = mp.loan_name 
       and mp.product = 'consumer_credit'
    left join private.californium_credit_score_audit_log c 
        on c.user_id = mum.user_id 
       and c.requested_on <= m.creation_date
    left join rating_map rm
        on c.rating = rm.rating
    where c.rating <> 'UNKNOWN'
),
last_bad as (
    -- find last N/O/P per user
    select user_id, max(requested_on) as last_bad_date
    from scored
    where rating in ('N','O','P')
    group by user_id
),
last_change_to_good as (
    -- find the first good score (A–M) *after* last_bad_date
    select s.user_id, min(s.requested_on) as last_change_to_good_date
    from scored s
    join last_bad lb
      on s.user_id = lb.user_id
     and s.requested_on > lb.last_bad_date
     and s.rating not in ('N','O','P')
    group by s.user_id
)
select
    l.user_id,
    l.encoded_key,
    l.rating as latest_rating,
    l.requested_on as latest_requested_on,
    w.rating as worst_rating,
    w.requested_on as worst_requested_on,
    g.last_change_to_good_date,
    l.purpose,
    l.creation_date,
    l.loan_amount
from scored l
join scored w
  on l.user_id = w.user_id
 and w.rn_worst = 1
left join last_change_to_good g
  on l.user_id = g.user_id
where l.rn_latest = 1

"""

schufa_score_data = pd.read_sql_query(query, conn)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time/60, 'minutes')

/var/folders/1f/pct68c2j21dd7jgky0y_thb80000gp/T/ipykernel_86402/320777742.py:91: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schufa_score_data = pd.read_sql_query(query, conn)


Execution time: 1.9392539819081624 minutes


In [584]:
schufa_score_data['creation_date'] = pd.to_datetime(schufa_score_data['creation_date'])

In [571]:
merged_v2 = merged_v1.merge(schufa_score_data[['user_id', 'latest_rating', 'latest_requested_on', 'worst_rating', 'worst_requested_on']], how='left', on = ['user_id'])

In [572]:
merged_v2

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur,profit_eur,latest_rating,latest_requested_on,worst_rating,worst_requested_on
0,8a868e099175234a0191d60079d965d2,e358f2ec-ccbd-4880-9bf0-d55086c5072f,consumer_credit,4000.0,2024-09-09,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,204.71,204.71,G,2024-09-03,G,2024-09-03
1,8a868e13920c1a780192187747750bf7,6135ce49-c5f8-418d-b5ae-e6ac49708dc5,consumer_credit,3000.0,2024-09-22,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,80.73,80.73,F,2023-11-03,F,2023-11-03
2,8a868e278dc345c9018dc86074f029c7,a7762b7c-1a1e-4c64-9947-d96a01ca498b,consumer_credit,1000.0,2024-02-20,NaT,NaN,2024-02-29 00:00:00,NaT,False,0.0,42.16,42.16,B,2024-02-20,P,2022-03-11
3,8a868e2a946bbfb0019474ab6d5132d3,6637abb5-8c4e-4efe-863a-4b3ff9ec3036,consumer_credit,1100.0,2025-01-17,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,56.00,56.00,E,2024-09-10,M,2022-04-29
4,8a868e2a94f6d4180194f92d38636bb3,f41a88b3-ae36-4fc0-a5f0-f99ffe0de06e,consumer_credit,8000.0,2025-02-12,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,236.40,236.40,M,2021-03-01,M,2021-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12348,8a868fe18e65d8dc018e6cad949d44df,8af0f60e-8605-4974-b60a-a214e262d33e,consumer_credit,3000.0,2024-03-23,NaT,NaN,2024-03-31 00:00:00,NaT,False,0.0,112.44,112.44,E,2024-03-23,G,2023-12-15
12349,8a868ff0936f6afc01937c21de9178bf,082ec984-f6ff-4dce-addd-236fa63a1f4d,consumer_credit,4000.0,2024-11-30,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,103.54,103.54,G,2024-11-09,M,2023-06-04
12350,8a868ffb9175234e0191acccfc214206,20314e4f-1a79-468e-b3be-58d817b05ac4,consumer_credit,25000.0,2024-09-01,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,1621.25,1621.25,M,2024-09-01,M,2024-09-01
12351,8a868fff94dd10cf0194dde9bafa764a,255e8dd5-a750-446f-ba50-04f1ef06e800,consumer_credit,8000.0,2025-02-07,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,563.94,563.94,M,2022-09-19,M,2022-09-19


In [573]:
merged_v2['revenue_eur'].sum()

3549545.3099999996

In [574]:
recovered_defaults = merged_v2[merged_v2['worst_rating'].isin(['N','O','P'])]
recovered_defaults['write_off_dt_eom'] = recovered_defaults['write_off_dt'] + MonthEnd(0)

/var/folders/1f/pct68c2j21dd7jgky0y_thb80000gp/T/ipykernel_86402/2103616186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recovered_defaults['write_off_dt_eom'] = recovered_defaults['write_off_dt'] + MonthEnd(0)


In [575]:
recovered_defaults[recovered_defaults['worst_rating'].isin(['P'])]['eur_written_off_within_6m'].sum()

40078.0

In [548]:
recovered_defaults[recovered_defaults['worst_rating'].isin(['P'])]

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur,profit_eur,latest_rating,latest_requested_on,worst_rating,worst_requested_on
2,8a868e278dc345c9018dc86074f029c7,a7762b7c-1a1e-4c64-9947-d96a01ca498b,consumer_credit,1000.0,2024-02-20,NaT,NaN,2024-02-29 00:00:00,NaT,False,0.0,42.16,42.16,B,2024-02-20,P,2022-03-11
19,8a868ea69389f73d0193deb1eefd2da8,0fbc65be-7400-44e0-9762-669f0a236fb3,consumer_credit,5000.0,2024-12-19,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,325.04,325.04,F,2024-11-24,P,2021-11-17
21,8a868eb494dd10cf0194ea1706ce20f0,d2604160-d5f6-414e-bc2b-f0b3d9182107,consumer_credit,10000.0,2025-02-09,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,658.88,658.88,G,2025-01-24,P,2024-11-15
27,8a868ede946bbfa501948e414f7c1b32,26f5cca7-8f72-4d21-a765-e86b238fbdf5,consumer_credit,7500.0,2025-01-22,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,451.02,451.02,M,2024-11-21,P,2024-08-01
54,8a868f4390918bc20190c53c34f34660,b4d765c9-bc75-4156-be84-7711b5cafffb,consumer_credit,5000.0,2024-07-18,NaT,NaN,2024-07-31 00:00:00,NaT,False,0.0,284.35,284.35,D,2024-07-18,P,2020-05-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12241,8a868ff0931c67f501932c60b1141552,c3e7f273-761a-4114-b7fb-4984887b6c44,consumer_credit,2000.0,2024-11-14,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,117.65,117.65,D,2024-04-16,P,2022-08-02
12254,8a868e1e948d081e01948ef1e8bd303c,bbf968a7-bc35-46c9-94c9-91440c0fea2e,consumer_credit,20000.0,2025-01-22,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,610.36,610.36,F,2024-09-03,P,2022-08-17
12285,8a868eb69503841a0195038530430843,2ceb86e7-32a3-4a96-ae0c-e09981eae7b4,consumer_credit,3600.0,2025-02-14,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,232.26,232.26,E,2024-10-23,P,2020-08-28
12298,8a868ee090705d680190988ac53372c7,752ccf51-ece8-43df-bba9-e0b9a6a923f4,consumer_credit,10000.0,2024-07-09,NaT,NaN,2024-07-31 00:00:00,NaT,False,0.0,555.63,555.63,C,2024-07-09,P,2017-06-12


In [552]:
recovered_defaults


,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur,profit_eur,latest_rating,latest_requested_on,worst_rating,worst_requested_on
2,8a868e278dc345c9018dc86074f029c7,a7762b7c-1a1e-4c64-9947-d96a01ca498b,consumer_credit,1000.0,2024-02-20,NaT,NaN,2024-02-29 00:00:00,NaT,False,0.0,42.16,42.16,B,2024-02-20,P,2022-03-11
19,8a868ea69389f73d0193deb1eefd2da8,0fbc65be-7400-44e0-9762-669f0a236fb3,consumer_credit,5000.0,2024-12-19,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,325.04,325.04,F,2024-11-24,P,2021-11-17
21,8a868eb494dd10cf0194ea1706ce20f0,d2604160-d5f6-414e-bc2b-f0b3d9182107,consumer_credit,10000.0,2025-02-09,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,658.88,658.88,G,2025-01-24,P,2024-11-15
22,8a868eb6934b5bbc0193553808637625,b5036088-91bd-4c84-92cb-2e3b3919dc94,consumer_credit,9800.0,2024-11-22,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,589.57,589.57,E,2024-09-14,O,2023-04-23
27,8a868ede946bbfa501948e414f7c1b32,26f5cca7-8f72-4d21-a765-e86b238fbdf5,consumer_credit,7500.0,2025-01-22,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,451.02,451.02,M,2024-11-21,P,2024-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12288,8a868ebf929c47540192b4460a2c6aa0,dfb43f7f-f05a-4759-846f-16ffef6d25a6,consumer_credit,1500.0,2024-10-22,NaT,NaN,2024-10-31 00:00:00,NaT,False,0.0,73.68,73.68,C,2023-04-22,O,2023-04-19
12289,8a868ebf94dd0fa30194df6b7a43583c,de25c25b-4d8a-4810-847b-c88108d37c46,consumer_credit,7000.0,2025-02-07,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,429.99,429.99,M,2024-11-15,O,2020-08-13
12298,8a868ee090705d680190988ac53372c7,752ccf51-ece8-43df-bba9-e0b9a6a923f4,consumer_credit,10000.0,2024-07-09,NaT,NaN,2024-07-31 00:00:00,NaT,False,0.0,555.63,555.63,C,2024-07-09,P,2017-06-12
12313,8a868f2194f6d427019513fff4dc21dc,5821745c-792b-4b40-8bdd-2d258880d2a5,consumer_credit,1000.0,2025-02-17,NaT,NaN,2025-02-28 00:00:00,NaT,False,0.0,42.93,42.93,M,2024-03-23,O,2024-02-03


In [553]:
recovered_defaults.groupby('write_off_dt_eom')['eur_written_off_within_6m'].sum()

write_off_dt_eom
2024-08-31        0.0
2024-11-30    15799.0
2024-12-31        0.0
2025-01-31     6508.0
2025-02-28     4956.0
2025-03-31    22927.0
2025-04-30    16024.0
2025-05-31    10396.0
2025-06-30    16134.0
2025-07-31        0.0
2025-08-31        0.0
2025-09-30        0.0
Name: eur_written_off_within_6m, dtype: float64

In [474]:
st = time.time()

query = f"""

select user_id, instrument_id, onbalance_ead,
case when dpd = 0 then '0'
     when dpd <= 30 then '1_30'
     when dpd <= 50 then '31_50'
     when dpd <= 60 then '51_60'
     when dpd <= 70 then '61_70'
     when dpd <= 80 then '71_80'
     when dpd <= 90 then '81_90'
     else '90+'
     end dpd_bucket
from credit_risk_playground.bp_retail_provisions_m
where reportdate = '2025-08-31'
and product_flag = 'CC'

"""

data_arrears = pd.read_sql_query(query, conn)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time/60, 'minutes')

/var/folders/1f/pct68c2j21dd7jgky0y_thb80000gp/T/ipykernel_86402/4011292671.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_arrears = pd.read_sql_query(query, conn)


Execution time: 0.00543363094329834 minutes


In [475]:
current_arrears = data_arrears[data_arrears['dpd_bucket'] != '0']

In [476]:
current_arrears_schufa = current_arrears.merge(schufa_score_data, on = 'user_id', how='inner')

In [479]:
current_arrears_schufa[((current_arrears_schufa['worst_rating'].isin(['N','O','P'])) & (current_arrears_schufa['dpd_bucket'].isin(['90+', '81_90','71_80'])))]['onbalance_ead'].sum()

73699.22

In [380]:
final = merged_v1.merge(schufa_score_data[['user_id','latest_rating', 'latest_requested_on', 'worst_rating', 'worst_requested_on','last_change_to_good_date']], on=['user_id'], how='left')

In [381]:
final

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur,profit_eur,latest_rating,latest_requested_on,worst_rating,worst_requested_on,last_change_to_good_date
0,8a868e0a9389ee1601938df2c4372061,c709993e-0f3b-415a-9e2f-a29215d916c7,consumer_credit,5000.0,2024-12-03,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,305.14,305.14,F,2024-10-17,F,2024-10-17,None
1,8a868e0a9389ee160193914364f00c54,363a69bd-b5ea-46bd-b7da-052b199f061f,consumer_credit,2000.0,2024-12-04,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,112.44,112.44,M,2024-11-04,O,2023-01-24,2023-03-04
2,8a868e0a9389ee160193b04e0c613596,5f8bf450-e413-44fc-9f6d-78b03b28c6c8,consumer_credit,1000.0,2024-12-10,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,70.14,70.14,M,2024-04-04,P,2022-03-15,2024-01-11
3,8a868e0d90dff2ba0190fef0be9d2373,c9131751-b272-4ff7-a583-67640cbde222,consumer_credit,1500.0,2024-07-29,NaT,NaN,2024-07-31 00:00:00,NaT,False,0.0,69.90,69.90,C,2024-07-29,E,2023-11-23,None
4,8a868e0d931c681101932029fff2641a,5875ac37-1f00-4a98-bf21-31c553d80c78,consumer_credit,3200.0,2024-11-12,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,184.08,184.08,M,2024-10-29,M,2024-10-29,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9755,8a868fdc9175234e0191fca2ff20392e,0eb1227a-4c99-433b-891b-a937b56d16c4,consumer_credit,8000.0,2024-09-16,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,381.26,381.26,F,2024-08-31,F,2024-08-31,None
9756,8a868fef8f2c7a05018f575379635e01,fb3947ba-2b78-4685-820b-30e84df6c3bc,consumer_credit,6500.0,2024-05-08,2025-06-02,1356.0,2024-05-31 00:00:00,2025-06-30,False,0.0,290.82,290.82,C,2024-05-05,F,2022-02-21,None
9757,8a868ff0944292070194495c02063cab,a10f2e60-2c38-47d4-a2ee-d0a7a8b0251e,consumer_credit,5000.0,2025-01-09,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,196.36,196.36,M,2024-11-06,M,2024-11-06,None
9758,8a868ff5934b5baa0193581bfc155563,22fc5feb-6116-4235-8672-c5ad1ae939b8,consumer_credit,5000.0,2024-11-23,NaT,NaN,2024-11-30 00:00:00,NaT,False,0.0,120.97,120.97,G,2024-11-19,G,2024-11-19,None


In [389]:
final[final['worst_rating'].isin(['P'])]['eur_written_off_within_6m'].sum() / final[final['worst_rating'].isin(['P'])]['revenue_eur'].sum()

0.3547194546128069

In [388]:
final[final['worst_rating'].isin(['P'])]['eur_written_off_within_6m'].sum()

83069.0

In [319]:
final["last_change_to_good_date"] = pd.to_datetime(final["last_change_to_good_date"])
final["creation_date"] = pd.to_datetime(final["creation_date"])

final["months_diff"] = (
    (final["creation_date"].dt.year - final["last_change_to_good_date"].dt.year) * 12 +
    (final["creation_date"].dt.month - final["last_change_to_good_date"].dt.month)
)

In [410]:
final[final['worst_rating'].isin(['N','O','P'])]

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_6m,eur_written_off_within_6m,revenue_eur,profit_eur,latest_rating,latest_requested_on,worst_rating,worst_requested_on,last_change_to_good_date
1,8a868e0a9389ee160193914364f00c54,363a69bd-b5ea-46bd-b7da-052b199f061f,consumer_credit,2000.0,2024-12-04,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,112.44,112.44,M,2024-11-04,O,2023-01-24,2023-03-04
2,8a868e0a9389ee160193b04e0c613596,5f8bf450-e413-44fc-9f6d-78b03b28c6c8,consumer_credit,1000.0,2024-12-10,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,70.14,70.14,M,2024-04-04,P,2022-03-15,2024-01-11
7,8a868e26936f6bf201938748149258e1,efae3399-9eee-4345-a05f-b8fe7cdb53a6,consumer_credit,4620.0,2024-12-02,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,328.16,328.16,M,2024-09-18,N,2023-05-26,2024-09-18
18,8a868e96938a01460193c1a6c108664d,6e24022b-ac6a-4b5a-839e-e95dd62a24a6,consumer_credit,6000.0,2024-12-13,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,359.84,359.84,C,2020-09-28,P,2020-01-05,2020-09-28
19,8a868e96938a014601943a74847f312b,1864ed22-c2ac-4e25-ae81-7193512a33be,consumer_credit,1000.0,2025-01-06,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,55.67,55.67,M,2024-11-16,O,2024-01-19,2024-03-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9710,8a868e698f0a3795018f1b2bc9795eb4,fd322cdc-2611-4a31-a88a-184c1e0b35f4,consumer_credit,9000.0,2024-04-26,NaT,NaN,2024-04-30 00:00:00,NaT,False,0.0,454.28,454.28,C,2024-04-26,O,2022-08-13,2024-04-06
9716,8a868ea59389ee150193b9eed3735aff,381b76b9-34d1-4e2f-80e9-6fe90dba65c7,consumer_credit,1000.0,2024-12-12,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,61.38,61.38,G,2024-11-08,O,2024-10-01,2024-11-08
9719,8a868eaa91b113c901920b80112104f4,8e2c6d15-4b4c-4215-b4e8-f4d7edbff7d5,consumer_credit,1000.0,2024-09-19,NaT,NaN,2024-09-30 00:00:00,NaT,False,0.0,57.63,57.63,D,2024-09-02,O,2020-12-24,2021-01-28
9735,8a868f1f944291c001944d3c54ee1744,83f6603e-7fbd-4c74-98bf-22beb8216551,consumer_credit,2000.0,2025-01-09,NaT,NaN,2025-01-31 00:00:00,NaT,False,0.0,141.36,141.36,E,2024-09-19,P,2024-01-13,2024-09-17


In [328]:
final[((final['worst_rating'].isin(['O','P'])) & (final['months_diff']<=48))]['revenue_eur'].sum()

286629.86

In [329]:
final[((final['worst_rating'].isin(['O','P'])) & (final['months_diff']<=48))]['eur_written_off_within_8m'].sum()

126349.0

In [330]:
pivot = pd.pivot_table(
    final,
    index='latest_rating',
    columns='worst_rating',
    values='eur_written_off_within_8m',
    aggfunc='sum',
    fill_value=0
)

pivot

worst_rating,A,B,C,D,E,F,G,H,I,K,L,M,N,O,P
latest_rating,,,,,,,,,,,,,,,
A,4901.0,993.0,0.0,924.0,0.0,18395.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8535.0
B,0.0,0.0,0.0,1785.0,1708.0,4283.0,0.0,0.0,0.0,0.0,0.0,35011.0,0.0,2376.0,6508.0
C,0.0,0.0,6797.0,6500.0,10170.0,7882.0,2258.0,0.0,4590.0,0.0,0.0,6349.0,0.0,7299.0,19714.0
D,0.0,0.0,0.0,4666.0,0.0,0.0,12996.0,0.0,0.0,0.0,0.0,15008.0,0.0,0.0,15908.0
E,0.0,0.0,0.0,0.0,53977.0,0.0,0.0,0.0,0.0,0.0,0.0,34879.0,0.0,0.0,6038.0
F,0.0,0.0,0.0,0.0,0.0,10025.0,11422.0,0.0,0.0,0.0,0.0,24884.0,0.0,0.0,901.0
G,0.0,0.0,0.0,0.0,0.0,0.0,25614.0,0.0,0.0,0.0,0.0,14449.0,0.0,0.0,24914.0
M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,134316.0,0.0,0.0,34156.0
O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [332]:
final[final['user_id']=='ce0bf2b4-143a-4799-b992-53f80bf9e8fa']

,encoded_key,user_id,product,loan_amount,creation_date,write_off_dt,eur_written_off,creation_date_eom,write_off_dt_eom,writeoff_within_8m,eur_written_off_within_8m,revenue_eur,profit_eur,latest_rating,latest_requested_on,worst_rating,worst_requested_on,last_change_to_good_date,months_diff
1055,8a868f149389f7420193a71b56b21d93,ce0bf2b4-143a-4799-b992-53f80bf9e8fa,consumer_credit,25000.0,2024-12-08,NaT,NaN,2024-12-31 00:00:00,NaT,False,0.0,1807.32,1807.32,G,2024-11-16,M,2024-09-29,NaT,NaN
